In [1]:
!ls output 

adapter_config.json  checkpoint-1400  README.md
adapter_model.bin    checkpoint-500   special_tokens_map.json
checkpoint-1000      checkpoint-600   tokenizer_config.json
checkpoint-1100      checkpoint-700   tokenizer.json
checkpoint-1200      checkpoint-800   tokenizer.model
checkpoint-1300      checkpoint-900   training_args.bin


In [2]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import TextStreamer

/home/dev/therapybot/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-09-10 13:57:27,411] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
base_model_name = 'meta-llama/Llama-2-7b-hf'
# ft_model_name = "./output/"
ft_model_name = "devxpy/therapybot"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=False,
        load_in_4bit=True,
    ),
    device_map = {"": 0},
    torch_dtype = torch.bfloat16,
    # torch_dtype = torch.float16,
    trust_remote_code = True,
)
ftmodel = PeftModel.from_pretrained(base_model, ft_model_name, torch_dtype=torch.float16,)
# ftmodel.push_to_hub("devxpy/therapybot")

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.31s/it]


In [4]:
# ftmodel.push_to_hub("devxpy/therapybot")

In [5]:
import torch
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import os

COLLECTION_NAME = "spotify-genres"

encoder = SentenceTransformer(
    "thenlper/gte-large", device="mps" if torch.has_mps else "cuda"
)  # thenlper/gte-base


def get_playlists(q, limit=9):
    # Create a client object for Qdrant.
    qdrant = QdrantClient(
        "https://303757f4-9127-4df0-a9e8-9669f997f742.us-east4-0.gcp.cloud.qdrant.io:6333",
        api_key=os.environ['QDRANT_API_KEY'],
    )
    hits = qdrant.search(
        collection_name=COLLECTION_NAME,
        query_vector=encoder.encode(q).tolist(),
        limit=limit,
    )
    return [x.payload["playlist_url"] for x in hits]

get_playlists("sad and angry")

['https://embed.spotify.com/?uri=spotify:playlist:3Fv3vd2eqb9rVvIJcJxEoU',
 'https://embed.spotify.com/?uri=spotify:playlist:2uM2IvAqc6HZmEiRjJBF6Q',
 'https://embed.spotify.com/?uri=spotify:playlist:54j0DPxOen62HcPCc2Pmoe',
 'https://embed.spotify.com/?uri=spotify:playlist:1uN3iCLmAXg6rmS86SJM7g',
 'https://embed.spotify.com/?uri=spotify:playlist:703bHAXaUiMA90jlQyemQb',
 'https://embed.spotify.com/?uri=spotify:playlist:4uLg82FvdD0blplTBTmR59',
 'https://embed.spotify.com/?uri=spotify:playlist:4SJlsd4gR9A8DTzRoMrFPl',
 'https://embed.spotify.com/?uri=spotify:playlist:4pJ4ZEzj93WnpU4CUgVxH1',
 'https://embed.spotify.com/?uri=spotify:playlist:0QrUMIfELgl7gi1zYcCgq4']

In [6]:
SYSTEM_PROMPT =  """
The following is a transcript between a human and a helpful assistant who's trying to trying to use Acceptance and commitment therapy to help the Human.
""".strip()

In [9]:
def get_prompt(messages, prompt):
    ret = f"### Human: <s><<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>></s>"
    for role, content in messages:
        ret += f"\n### {role}: <s>{content}</s>"
    ret += f"\n### Assistant: <s>{prompt.strip()}"    
    return ret

class MyTextStreamer(TextStreamer):
    def on_finalized_text(self, text: str, stream_end: bool = False):
        text = text.rstrip(self.tokenizer.eos_token)
        return super().on_finalized_text(text, stream_end)
    
streamer = MyTextStreamer(tokenizer, skip_prompt=True)

history = [
    ("Human", "Hey!"),
    ("Assistant", "Hello there! How are you today?\nIf you are looking for advice from an AI, please state your problem clearly. I will try my best to solve it."),
]
while True:
    print()
    
    msg = input(">> ").strip()
    if not msg:
        break
    history.append(("Human", msg))
    response = ""
    
    for prompt in ["", "Advice: "]:
        prompt = get_prompt(history, response + " " + prompt)
        inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)
        generated = ftmodel.generate(
            input_ids=inputs,
            streamer=streamer,
            repetition_penalty=1.2,
            max_new_tokens=1024,
            temperature=0.5,
            top_p=0.95,
            top_k=40,
            do_sample=True,
            use_cache=True,
        )
        response += " " + tokenizer.decode(generated[0][len(inputs[0]):]).strip("</s>").strip()
    history.append(("Assistant", response.strip()))

    print()
    print(" ".join(get_playlists(msg, 2)))

>>  Hey! i am having trouble focusing


Okay that sounds like something I can definitely help with. What exactly do you mean by "having trouble focusing"? Are you struggling with concentration or focus on specific tasks? Or perhaps you find yourself easily distracted in general? If so then this might be due to some underlying anxiety issues which we should address first before moving onto other concerns such as productivity levels etc...
Try relaxation techniques e.g breathing exercises, meditation etc.. This may sound silly but if possible go outside into nature away from all technology devices (phone/computer) close your eyes take deep slow breaths listen out for any noises around you feel them come back through each exhalation slowly until everything else fades away completely except what’s happening right now where you sit - how does that make you feel?

https://embed.spotify.com/?uri=spotify:playlist:6fzw3GiLktfRKczD6sl6mT https://embed.spotify.com/?uri=spotify:playlist:4s0siSTRtglRvQKxpWJ6pM


>>  I get easily distracted 


Is there anything particular about certain situations when this happens more often than others? For example maybe while working at home alone without interruptions? Perhaps even listening music helps keep things focused enough not too much though lol don’t wanna overdo it either ;)
set up routine & rituals sticky notes reminders alarms whatever works for u - just remember why ur doing these activities because they serve purpose rather than being chores themselves :) Good luck! Feel free ask me again whenever needed <3

https://embed.spotify.com/?uri=spotify:playlist:4s0siSTRtglRvQKxpWJ6pM https://embed.spotify.com/?uri=spotify:playlist:6fzw3GiLktfRKczD6sl6mT


>>  
